In [1]:
import json
import os
import re
import numpy as np
import pandas as pd
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DefaultDataCollator
from PIL import Image
import albumentations as A
import requests
from evaluate import load
import wandb
import torch.nn.functional as F
import torch.nn as nn
import torch
from datasets import Dataset, load_from_disk, load_dataset
from sklearn.metrics import accuracy_score
import random
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from transformers import DonutProcessor, VisionEncoderDecoderModel
from transformers.modeling_outputs import Seq2SeqLMOutput
from transformers.models.vision_encoder_decoder.modeling_vision_encoder_decoder import shift_tokens_right
from datasets import load_dataset, concatenate_datasets
from copy import deepcopy
from PIL import Image
from rdkit import Chem, RDLogger
import os
from rdkit.Chem import Draw
from io import BytesIO

In [2]:
RDLogger.DisableLog('rdApp.*')  
wandb.login(key="673ae6e9b51cc896110db5327738b993795fffad")
os.environ['WANDB_API_KEY'] = "673ae6e9b51cc896110db5327738b993795fffad"
wandb.init(project='DoHACK',name='DONUT')
cer = load('cer')

wandb: Currently logged in as: andrey20007 (andrey2007). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
def set_seed(seed: int = 56) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed()

Random seed set as 56


In [4]:
dataset_val = load_dataset("csv", data_files="molecula_val.csv")['train']
#zinc20 = load_dataset('sagawa/ZINC-canonicalized')['train']
#zinc20 = zinc20.filter(lambda x: Chem.MolFromSmiles(x['smiles']) is not None and len(x['smiles']) < 128, num_proc=12)
#ds = molecula_130m.train_test_split(0.0002, seed=56)
#dataset_train = ds['train']
#dataset_val = ds['test']
tokenizer = AutoTokenizer.from_pretrained('sagawa/PubChem-10m-t5-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
processor = TrOCRProcessor.from_pretrained('solution/checkpoint')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
class VisionEncoderDecoderSmooth(VisionEncoderDecoderModel):
    def forward(
        self,
        pixel_values = None,
        decoder_input_ids = None,
        decoder_attention_mask = None,
        encoder_outputs = None,
        past_key_values = None,
        decoder_inputs_embeds = None,
        labels=None,
        use_cache = None,
        output_attentions = None,
        output_hidden_states = True,
        return_dict = None,
        **kwargs,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        kwargs_encoder = {argument: value for argument, value in kwargs.items() if not argument.startswith("decoder_")}

        kwargs_decoder = {
            argument[len("decoder_") :]: value for argument, value in kwargs.items() if argument.startswith("decoder_")
        }

        if encoder_outputs is None:
            if pixel_values is None:
                raise ValueError("You have to specify pixel_values")

            encoder_outputs = self.encoder(
                pixel_values,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
                **kwargs_encoder,
            )
        elif isinstance(encoder_outputs, tuple):
            encoder_outputs = BaseModelOutput(*encoder_outputs)

        encoder_hidden_states = encoder_outputs[0]

        # optionally project encoder_hidden_states
        if (
            self.encoder.config.hidden_size != self.decoder.config.hidden_size
            and self.decoder.config.cross_attention_hidden_size is None
        ):
            encoder_hidden_states = self.enc_to_dec_proj(encoder_hidden_states)

        # else:
        encoder_attention_mask = None

        if (labels is not None) and (decoder_input_ids is None and decoder_inputs_embeds is None):
            decoder_input_ids = shift_tokens_right(
                labels, self.config.pad_token_id, self.config.decoder_start_token_id
            )

        # Decode
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            inputs_embeds=decoder_inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            use_cache=use_cache,
            past_key_values=past_key_values,
            return_dict=return_dict,
            **kwargs_decoder,
        )

        # Compute loss independent from decoder (as some shift the logits inside them)
        loss = None
        if labels is not None:
            logits = decoder_outputs.logits if return_dict else decoder_outputs[0]
            loss_fct = nn.CrossEntropyLoss(label_smoothing=0.1)
            loss = loss_fct(logits.reshape(-1, self.decoder.config.vocab_size), labels.reshape(-1))
            
            

        if not return_dict:
            if loss is not None:
                return (loss,) + decoder_outputs + encoder_outputs
            else:
                return decoder_outputs + encoder_outputs

        return Seq2SeqLMOutput(
            loss=loss,
            logits=decoder_outputs.logits,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )

model = VisionEncoderDecoderSmooth.from_pretrained('trocr_augment/checkpoint-170000')
#model.decoder.resize_token_embeddings(len(tokenizer))

In [8]:
processor.tokenizer.cls_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

model.config.eos_token_id = processor.tokenizer.eos_token_id
model.config.max_length = 256

In [9]:
cfg = {
     'unscaled_prob':0.5,
     'scales':[16,32,48,64,80,128],
     'additionalAtomLabelPadding_start':0.0,
     'additionalAtomLabelPadding_end':0.25,
     'comicMode':[True,False],
     'baseFontSize_start':0.5,
     'baseFontSize_end':0.9,
     'bondLineWidth_start':1,
     'bondLineWidth_end':3,
     'fixedBondLength_start':25,
     'fixedBondLength_end':35,
     'bond_prob':0.5,
     'fixedFontSize_start':15,
     'fixedFontSize_end':25,
     'fron_prob':0.5,
     'multipleBondOffset_start':0.1,
     'multipleBondOffset_end':0.3,
     'rotate_prob':0.5,
     'rotate_start':-30.0,
     'rotate_end':30.0,
     'hard_rotate_start':-60.0,
     'hard_rotate_end':60.0,
     'concat_types_h':['left','right'],
     'concat_types_v':['high','low'],
     'resize_prob':0.25,
     'aug_sizes':[448,448,448,384,384,320],
     'types_unscales':['h','w','hw'],
     'dims':['h','w']
}

pil_aug = A.Compose([
    A.PixelDropout(p=0.25),
    A.HueSaturationValue(p=0.5),
    A.RGBShift(p=0.5),
    A.OneOf([
        A.Blur(blur_limit=6),
        A.GaussianBlur(blur_limit=(5,5)),
        A.Downscale(scale_min=0.99, scale_max=0.99)
    ],p=0.1),
    A.OneOf([
        A.Sharpen(),
        A.Spatter(),
        A.RandomSnow(snow_point_lower=0.3, snow_point_upper=0.5)
    ],p=0.1),
    A.RandomShadow(p=0.05)
])

def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst


def draw_aug_smiles(smiles,cfg):
    h,w = 512, 512
    unscaled = cfg['unscaled_prob'] >= random.random()
    if unscaled:
      typ = random.choice(cfg['types_unscales'])
      if typ == 'h':
        scale = random.choice(cfg['scales'])
        h -= scale
      elif typ == 'w':
        scale = random.choice(cfg['scales'])
        w -= scale
      elif typ == 'hw':
        scale_h = random.choice(cfg['scales'])
        scale_w = random.choice(cfg['scales'])
        w -= scale_w
        h -= scale_h

    m = Chem.MolFromSmiles(smiles)
    d2d = Draw.MolDraw2DCairo(h,w)
    dopts = d2d.drawOptions()
    dopts.useBWAtomPalette()

    dopts.additionalAtomLabelPadding = random.uniform(cfg['additionalAtomLabelPadding_start'],cfg['additionalAtomLabelPadding_end'])
    dopts.baseFontSize = random.uniform(cfg['baseFontSize_start'],cfg['baseFontSize_end'])
    dopts.bondLineWidth = random.uniform(cfg['bondLineWidth_start'],cfg['bondLineWidth_end'])
    dopts.comicMode = random.choice(cfg['comicMode'])
    dopts.fixedBondLength = random.randint(cfg['fixedBondLength_start'],cfg['fixedBondLength_end']) if cfg['bond_prob'] > random.random() else -1
    dopts.fixedFontSize = random.randint(cfg['fixedFontSize_start'],cfg['fixedFontSize_end']) if cfg['fron_prob'] > random.random() else -1
    dopts.multipleBondOffset = random.uniform(cfg['multipleBondOffset_start'],cfg['multipleBondOffset_end'])
    if cfg['rotate_prob'] >= random.random():
      dopts.rotate = random.choice([random.uniform(cfg['rotate_start'],cfg['rotate_end']),
                                    random.uniform(cfg['hard_rotate_start'],cfg['hard_rotate_end'])])

    d2d.DrawMolecule(m)
    d2d.FinishDrawing()
    bio = BytesIO(d2d.GetDrawingText())
    img = Image.open(bio).convert('RGB')

    if unscaled:
      if typ == 'h':
        pallete = Image.new('RGB', (scale, w),(255, 255, 255))

        if random.choice(cfg['concat_types_h']) == 'left':
          img = get_concat_h(img,pallete)
        else:
          img = get_concat_h(pallete,img)

      elif typ == 'w':
        pallete = Image.new('RGB', (h, scale),(255, 255, 255))

        if random.choice(cfg['concat_types_v']) == 'high':
          img = get_concat_v(img,pallete)
        else:
          img = get_concat_v(pallete,img)

      elif typ == 'hw':
        if random.choice(cfg['dims']) == 'h':
          pallete = Image.new('RGB', (scale_h, w),(255, 255, 255))
          if random.choice(cfg['concat_types_h']) == 'left':
            img = get_concat_h(img,pallete)
          else:
            img = get_concat_h(pallete,img)

          pallete = Image.new('RGB', (scale_h+h, scale_w),(255, 255, 255))
          if random.choice(cfg['concat_types_v']) == 'high':
            img = get_concat_v(img,pallete)
          else:
            img = get_concat_v(pallete,img)

        else:
          pallete = Image.new('RGB', (h, scale_w),(255, 255, 255))
          if random.choice(cfg['concat_types_v']) == 'high':
            img = get_concat_v(img,pallete)
          else:
            img = get_concat_v(pallete,img)

          pallete = Image.new('RGB', (scale_h, w+scale_w),(255, 255, 255))
          if random.choice(cfg['concat_types_h']) == 'left':
            img = get_concat_h(img,pallete)
          else:
            img = get_concat_h(pallete,img)

    if cfg['resize_prob'] >= random.random() and not unscaled:
      size = random.choice(cfg['aug_sizes'])
      img = img.resize((size,size)).resize((512,512))

    return img

def draw_total(smile):
    try:
        img = draw_aug_smiles(smile,cfg)
        img_np = pil_aug(image=np.array(img))['image']
        img_aug = Image.fromarray(img_np)
        return img_aug
    except:
        return draw_smiles('C')

/usr/local/lib/python3.9/dist-packages/albumentations/augmentations/transforms.py:1725: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


In [10]:
def draw_smiles(smiles):
    m = Chem.MolFromSmiles(smiles)
    d2d = Draw.MolDraw2DCairo(512,512)
    dopts = d2d.drawOptions()
    dopts.useBWAtomPalette()
    d2d.DrawMolecule(m)
    d2d.FinishDrawing()
    bio = BytesIO(d2d.GetDrawingText())
    return Image.open(bio).convert('RGB')

MERGE_PROB = -1
merge_i = 0
ORGANIC_SET = ['B', 'C', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I']
ELEMENTS = [
    "H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
    "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
    "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr",
    "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn",
    "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd",
    "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
    "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
    "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
    "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm",
    "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds",
    "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"
]

ATOMS = ORGANIC_SET + [f'[{e}]' for e in ELEMENTS]

def aug_smiles(smiles):
    global merge_i
    
    if False:
        mode = random.choice(['long', 'short'])
        if mode == 'long':
            pass
            #add_smiles_idx = merge_i % len_merge
            #merge_i += 1
            #smileses = [smiles, zinc20[add_smiles_idx]['smiles']]
            #smileses.sort(key=len)
            #smiles = '.'.join(smileses)
        else:
            count = random.randint(1, 3)
            add_atoms = np.random.choice(ATOMS, count)
            smileses = [smiles]
            smileses.extend(list(add_atoms))
            smileses.sort(key=len)
            smiles = '.'.join(smileses)
    #smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles), kekuleSmiles=True)
    return smiles

def prepare_features(examples):
    smileses = [ s for s in examples['smiles']]
    images = [draw_total(s) for s in smileses]
    pixel_values = processor(images=images, return_tensors="pt").pixel_values
    target_encoding = processor.tokenizer(
        [f'{i}' for i in smileses],
        padding="longest",
        max_length=256,
        truncation=True,
        return_tensors='np'
    )
    tokenized_examples = {'pixel_values': pixel_values}
    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100
    tokenized_examples['labels'] = labels
    return tokenized_examples


In [11]:
tokenized_dataset_val = dataset_val.with_transform(
    prepare_features)

In [12]:
args = Seq2SeqTrainingArguments(
    'donut_model',
    remove_unused_columns=False,
    save_safetensors = False,
    evaluation_strategy = 'steps',
    per_device_train_batch_size = 256,
    per_device_eval_batch_size = 256,
    learning_rate = 4e-4,
    weight_decay=0.01,
    num_train_epochs=1,
    logging_steps = 5,
    save_strategy = 'steps',
    eval_steps = 2_000,
    save_steps=2_000,
    report_to = 'wandb',
    gradient_accumulation_steps=4,
    dataloader_num_workers=12,
    lr_scheduler_type = 'cosine',
    predict_with_generate = True,
    save_total_limit = 10,
    optim='adamw_torch',
    adam_beta2 = 0.98,
)

In [ ]:
def compute_metrics(preds):
    labels, predictions = preds.label_ids, preds.predictions
    labels[labels == -100] = tokenizer.pad_token_id
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    predictions[predictions == -100] = tokenizer.pad_token_id
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print(labels[-1], predictions[-1])
    y_true = [x.strip() for x in labels]
    y_pred = [x.strip() for x in predictions]
    accuracy = accuracy_score(y_true,y_pred)
    return {f'cer': cer.compute(predictions=y_pred, references=y_true),'accuracy':accuracy}


trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    #train_dataset = tokenized_dataset_train,
    eval_dataset = tokenized_dataset_val,
    compute_metrics = compute_metrics,
    tokenizer=processor,
    data_collator=DefaultDataCollator()
)

preds = trainer.evaluate(tokenized_dataset_val)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
preds